In [2]:
import numpy as np
import pandas as pd

import time
from datetime import datetime as dt

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go


# 1. Data Preparation

3 datasets:

S&P 500 daily adjusted price

Mean-variance weights table

Min-variance weights table

In [3]:
data_adj = pd.read_csv("data_processor_update/outputs/final_ratios.csv")
weight_meanv = pd.read_excel('mean_weighted.xlsx')
weight_minv = pd.read_excel('minimum_weighted.xlsx')
weight_equal=pd.read_excel('equally_weighted.xlsx')

In [4]:
data_adj

,Unnamed: 0,date,gvkey,tic,gsector,adj_close_q,y_return,EPS,BPS,DPS,cur_ratio,quick_ratio,cash_ratio,acc_rec_turnover,debt_ratio,debt_to_equity,pe,ps,pb
0,0,1999-12-01,126554,A,35.0,25.871546,0.932748,1.35,8.900000,0.000,2.104700,0.972635,0.000000,0.000000,0.378766,0.000000,0.000000,0.000000,0.000000
1,1,2000-03-01,126554,A,35.0,65.752098,-0.402034,0.30,9.924779,0.000,2.410256,1.381229,0.682148,4.529412,0.368791,0.000000,330.937500,16.162480,6.668914
2,2,2000-06-01,126554,A,35.0,43.985371,-0.165343,0.67,10.263735,0.000,2.313358,1.311528,0.447392,3.380625,0.365934,0.000000,285.887097,18.712701,8.634771
3,3,2000-09-01,126554,A,35.0,37.282143,-0.154825,1.01,10.820834,0.000,2.129084,1.143426,0.280080,2.972312,0.373706,0.000000,127.343750,7.852132,3.765884
4,4,2000-12-01,126554,A,35.0,31.934578,-0.324580,1.68,11.597529,0.000,2.050399,1.159173,0.361131,2.882326,0.375074,0.000000,70.170455,6.968765,3.993307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63426,63426,2023-12-01,13721,ZTS,35.0,176.838669,0.065484,3.94,11.050541,0.375,3.869403,1.872512,1.090796,4.976929,0.640295,1.780055,134.868217,37.150632,15.756437
63427,63427,2024-03-01,13721,ZTS,35.0,188.806396,-0.098840,5.08,10.901745,0.375,3.357861,1.770778,1.080466,4.882669,0.650637,1.862352,173.131579,40.861741,18.126206
63428,63428,2024-06-03,13721,ZTS,35.0,171.037384,0.065329,1.31,11.069117,0.432,3.331063,1.711891,1.034573,5.063418,0.647895,1.840063,129.167939,35.273723,15.304830
63429,63429,2024-09-03,13721,ZTS,35.0,182.584076,-0.035154,2.68,10.942857,0.432,3.451308,1.645520,0.875904,4.741866,0.649841,1.855847,126.540146,33.307726,15.861461


In [5]:
weight_meanv.head()

,Unnamed: 0,trade_date,gvkey,weights,predicted_return
0,0,2009-06-01,1045,0.00699,0.052594
1,1,2009-06-01,1161,0.00696,0.098717
2,2,2009-06-01,1230,0.00696,0.036450
3,3,2009-06-01,1661,0.00698,0.062174
4,4,2009-06-01,1704,0.00698,0.058399


In [6]:
weight_minv.head()

,Unnamed: 0,trade_date,gvkey,weights,predicted_return
0,0,2009-06-01,1045,0.00694,0.052594
1,1,2009-06-01,1161,0.00694,0.098717
2,2,2009-06-01,1230,0.00694,0.036450
3,3,2009-06-01,1661,0.00694,0.062174
4,4,2009-06-01,1704,0.00694,0.058399


In [7]:
weight_equal.head()

,Unnamed: 0,trade_date,gvkey,weights,predicted_return
0,0,2009-06-01,1045,0.006944,0.052594
1,1,2009-06-01,1161,0.006944,0.098717
2,2,2009-06-01,1230,0.006944,0.036450
3,3,2009-06-01,1661,0.006944,0.062174
4,4,2009-06-01,1704,0.006944,0.058399


In [8]:
#the length of the 3 tables should be the same
print(len(weight_meanv),
len(weight_minv),
len(weight_equal))


16425 16425 16425


## 1.1 Get stock name, trade date, daily date

Add 20170901 as tradedate_full


In [9]:
#get unique stock name from table
#stock names are the same between mean variance and minimum variance
#only the weights are different
stocks_name = list(set(weight_meanv['gvkey']))
stocks_name.sort()

In [10]:
len(stocks_name)

697

In [11]:
#get all daily date
#convert to list from pandas.core.series.Series
#use set to get unique values
daily = list(set(data_adj['date']))
#sort it
daily.sort()
print(type(data_adj['date']),type(daily))

<class 'pandas.core.series.Series'> <class 'list'>


In [12]:
len(daily)


116

In [13]:
#get unique trade date from table
tradedate = list(set(weight_meanv['trade_date']))
tradedate.sort()


In [14]:
tradedate

['2004-03-01',
 '2004-06-01',
 '2004-09-01',
 '2004-12-01',
 '2005-03-01',
 '2005-06-01',
 '2005-09-01',
 '2005-12-01',
 '2006-03-01',
 '2006-06-01',
 '2006-09-01',
 '2006-12-01',
 '2007-03-01',
 '2007-06-01',
 '2007-09-04',
 '2007-12-03',
 '2008-03-03',
 '2008-06-02',
 '2008-09-02',
 '2008-12-01',
 '2009-03-02',
 '2009-06-01',
 '2009-09-01',
 '2009-12-01',
 '2010-03-01',
 '2010-06-01',
 '2010-09-01',
 '2010-12-01',
 '2011-03-01',
 '2011-06-01',
 '2011-09-01',
 '2011-12-01',
 '2012-03-01',
 '2012-06-01',
 '2012-09-04',
 '2012-12-03',
 '2013-03-01',
 '2013-06-03',
 '2013-09-03',
 '2013-12-02',
 '2014-03-03',
 '2014-06-02',
 '2014-09-02',
 '2014-12-01',
 '2015-03-02',
 '2015-06-01',
 '2015-09-01',
 '2015-12-01',
 '2016-03-01',
 '2016-06-01',
 '2016-09-01',
 '2016-12-01',
 '2017-03-01',
 '2017-06-01',
 '2017-09-01',
 '2017-12-01',
 '2018-03-01',
 '2018-06-01',
 '2018-09-04',
 '2018-12-03',
 '2019-03-01',
 '2019-06-03',
 '2019-09-03',
 '2019-12-02',
 '2020-03-02',
 '2020-06-01',
 '2020-09-

In [15]:
tradedate_full = [x for x in tradedate if x >= '2004-12-01']

In [16]:
tradedate_full

['2004-12-01',
 '2005-03-01',
 '2005-06-01',
 '2005-09-01',
 '2005-12-01',
 '2006-03-01',
 '2006-06-01',
 '2006-09-01',
 '2006-12-01',
 '2007-03-01',
 '2007-06-01',
 '2007-09-04',
 '2007-12-03',
 '2008-03-03',
 '2008-06-02',
 '2008-09-02',
 '2008-12-01',
 '2009-03-02',
 '2009-06-01',
 '2009-09-01',
 '2009-12-01',
 '2010-03-01',
 '2010-06-01',
 '2010-09-01',
 '2010-12-01',
 '2011-03-01',
 '2011-06-01',
 '2011-09-01',
 '2011-12-01',
 '2012-03-01',
 '2012-06-01',
 '2012-09-04',
 '2012-12-03',
 '2013-03-01',
 '2013-06-03',
 '2013-09-03',
 '2013-12-02',
 '2014-03-03',
 '2014-06-02',
 '2014-09-02',
 '2014-12-01',
 '2015-03-02',
 '2015-06-01',
 '2015-09-01',
 '2015-12-01',
 '2016-03-01',
 '2016-06-01',
 '2016-09-01',
 '2016-12-01',
 '2017-03-01',
 '2017-06-01',
 '2017-09-01',
 '2017-12-01',
 '2018-03-01',
 '2018-06-01',
 '2018-09-04',
 '2018-12-03',
 '2019-03-01',
 '2019-06-03',
 '2019-09-03',
 '2019-12-02',
 '2020-03-02',
 '2020-06-01',
 '2020-09-01',
 '2020-12-01',
 '2021-03-01',
 '2021-06-

## 1.2 Get the daily price matrix based on stock_name

In [17]:
#30mins running time
##################match current all stocks to daily adjusted price table#######################
# backtest daily price matrix:
# covert original table to matrix
# rows = all stocks: 971, columns = all dates:7155 days
#
start_time = time.time()

balance_daily = pd.DataFrame(index = stocks_name, columns = daily)
balance_daily = balance_daily.fillna(np.nan)

for i in stocks_name:
    ind = np.where(data_adj['gvkey'] == i)[0]
    tmp = data_adj.loc[ind, :]
    tmp.index = range(tmp.shape[0])
    for j in daily:
        ind2 = np.where(tmp['date'] == j)[0]
        if(len(ind2) != 0):
            balance_daily.loc[i,j] = tmp.loc[ind2[0], 'adj_close_q']

elapsed_time = time.time() - start_time
print(elapsed_time)
writer2 = pd.ExcelWriter('balance_daily.xlsx')
writer2 = pd.ExcelWriter('balance_daily_user8.xlsx')

balance_daily.to_excel(writer2,'Sheet1')
writer2.close()
###############################################

C:\Users\misss\AppData\Local\Temp\ipykernel_40080\1696695376.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  balance_daily = balance_daily.fillna(np.nan)


6.6129655838012695


C:\Users\misss\AppData\Local\Temp\ipykernel_40080\1696695376.py:26: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  balance_daily.to_excel(writer2,'Sheet1')


In [18]:
balance_daily = pd.read_excel('balance_daily_user8.xlsx', index_col=0)


In [19]:
balance_daily

,1996-03-01,1996-06-03,1996-09-03,1996-12-02,1997-03-03,1997-06-02,1997-09-02,1997-12-01,1998-03-02,1998-06-01,...,2022-09-01,2022-12-01,2023-03-01,2023-06-01,2023-09-01,2023-12-01,2024-03-01,2024-06-03,2024-09-03,2024-12-02
1045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.930000,13.980000,16.030001,14.700000,14.680000,13.020000,15.660000,11.530000,10.600000,14.610000
1075,NaN,7.730066,8.389183,9.284263,9.478542,8.999684,10.055619,12.135516,12.712487,14.311364,...,67.808128,70.360054,67.065178,70.373497,71.014015,71.022690,65.187202,74.559937,86.479935,91.468803
1078,NaN,4.717293,5.064067,6.179262,6.434070,7.102882,7.082232,7.690752,8.631998,8.442722,...,99.775932,103.201340,96.848946,99.253922,99.759033,102.291161,116.250908,101.315460,113.526405,116.200218
1161,NaN,8.750000,6.250000,12.687500,17.937500,19.875000,19.437500,11.281250,11.125000,9.625000,...,82.330002,77.480003,78.290001,119.470001,109.449997,121.389999,202.639999,163.550003,136.940002,142.059998
1209,NaN,14.317203,13.027093,17.007317,17.958735,19.500549,20.691166,19.316851,20.720919,21.759985,...,234.675720,294.974731,273.939209,261.215149,286.679657,263.875183,229.880035,265.820160,271.884644,333.571838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,72.438545,75.924255,87.442062,79.818817,94.209686,90.294510,95.216087,92.683456,94.180946,82.050003
312009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.065766,10.765805,14.564113,12.180532,15.303938,18.491840,21.097977,26.536190,22.375109,15.710000
316056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.766258,111.007080,109.401260,102.390770,112.585030,107.092667,125.412460,119.613861,136.542542,140.399994
322985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11.751506,14.251513,11.460000,NaN


# 2. Function: calculate the rebalance matrix:

1. shares table
2. cost table
3. cash table
4. portfolio table
5. portfolio_cost table
6. portfolio_return table
7. portfolio_cumsum table

In [20]:
# set initial capital to 1 million and transaction percet to 0.1%

capital = 1000000
transaction_percent = 1/1000

In [21]:
weight_meanv.shape

(16425, 5)

In [22]:
#construct an empty matrix
#rows are stocks: 971 unique stocks
#columns are trade dates
#name the index as stock names

def cal_portfolio(stocks_name,tradedate, weight_table, capital, transaction_percent):


    balance_weight = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_weight = balance_weight.fillna(0)
    
    #loop through the weights table to fill the matrix
    #need weight tabele
    #0. weights table
    for i in range(weight_table.shape[0]):
        tics = weight_table.loc[i, 'gvkey']
        date = weight_table.loc[i, 'trade_date']
        w = weight_table.loc[i, 'weights']
        balance_weight.loc[tics, date] = w
        
    #1. shares table
    balance_share = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_share['initial'] = 0
    balance_share = balance_share.fillna(0)
    #2. costs table
    balance_cost = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_cost = balance_cost.fillna(0)
    #3. cash table
    balance_cash = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_cash = balance_cash.fillna(0)

    #4. portfolio without transaction costs table 
    portfolio = pd.DataFrame(index = range(1), columns = tradedate_full)
    portfolio.loc[0,tradedate[0]] = capital
    
    
    # calculate portfolio value without cost
    balance_price_full = []
    balance_price_full = balance_daily[tradedate_full]
    balance_price = balance_daily[tradedate]
    balance_price.replace([0, np.inf, -np.inf], np.nan, inplace=True)
    for j in range(len(tradedate) - 1):
        i = tradedate[j]
        #current capital
        cap = portfolio.loc[0, i]
        #
        balance_cash[i] = cap * balance_weight[i]
        balance_share[i] = list(np.array(balance_cash[i])
                                      / np.array(balance_price[i]))
        balance_share[i] = balance_share[i].fillna(0)
        balance_share[i] = balance_share[i].astype(int)
        #next quarter price * current share to calculate next quarter total account value
        a = balance_price_full[tradedate_full[j+1]].fillna(0)
        portfolio.loc[0, tradedate_full[j+1]] = np.dot(balance_share[i], a)

    balance_cost = abs(np.diff(balance_share, axis = 1))
    balance_cost = balance_cost * balance_price
    balance_cost = np.sum(balance_cost, axis = 0) * transaction_percent
    
    #5. portfolio with transaction cost table
    #calcualte portolio value with transaction cost
    balance_cost.index = tradedate_full
    portfolio_cost = []
    portfolio_cost = portfolio.loc[0,] - balance_cost
    portfolio_cost.index = tradedate_full
    portfolio_cost_value = portfolio_cost.copy()
    
    #6. portfolio return P&L table
    portfolio_cumsum = portfolio_cost_value.copy()
    portfolio_cumsum.index = range(len(portfolio_cumsum))
    tmp = portfolio_cumsum.shift(1)
    tmp[0] = 0
    tmp.index = tradedate_full

    portfolio_cumsum = portfolio_cost_value- tmp
    portfolio_cumsum.index = range(len(portfolio_cumsum))
    tmp.index = range(len(tmp))
    for i in range(len(portfolio_cumsum)):
        if(tmp[i] != 0):
            portfolio_cumsum[i] = portfolio_cumsum[i]/tmp[i]

    portfolio_cumsum.index = range(len(portfolio_cumsum))
    portfolio_cumsum[0] = 0
    portfolio_return=portfolio_cumsum.copy()



    # 7. portfolio accumulative return table
    initial = 0
    for i in range(portfolio_cumsum.shape[0]):
        portfolio_cumsum[i] = initial + portfolio_cumsum[i]
        initial = portfolio_cumsum[i]
    
    #1. shares table
    #2. costs table
    #3. cash table
    #4. portfolio without transaction costs table 
    #5. portfolio with transaction cost table
    #6. portfolio return P&L table
    #7. portfolio accumulative return table

    return(balance_share,
           balance_cost,
           balance_cash,
           portfolio,
           portfolio_cost,
           portfolio_return,
           portfolio_cumsum)


In [23]:
#calculate mean variance allocation portolio
(balance_share_meanv,
 balance_cost_meanv,
 balance_cash_meanv,
 portfolio_meanv,
 portfolio_cost_meanv,
 portfolio_return_meanv,
 portfolio_cumsum_meanv)=cal_portfolio(stocks_name,tradedate_full, weight_meanv, capital, transaction_percent)


C:\Users\misss\AppData\Local\Temp\ipykernel_40080\1845856610.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  balance_weight = balance_weight.fillna(0)
C:\Users\misss\AppData\Local\Temp\ipykernel_40080\1845856610.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.00699' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  balance_weight.loc[tics, date] = w
C:\Users\misss\AppData\Local\Temp\ipykernel_40080\1845856610.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.00864' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
 

In [24]:
sum(portfolio_return_meanv)

np.float64(173.11592824171572)

In [25]:
#calculate minimum variance allocation portolio
(balance_share_minv,
 balance_cost_minv,
 balance_cash_minv,
 portfolio_minv,
 portfolio_cost_minv,
 portfolio_return_minv,
 portfolio_cumsum_minv)=cal_portfolio(stocks_name,tradedate_full, weight_minv, capital, transaction_percent)

C:\Users\misss\AppData\Local\Temp\ipykernel_40080\1845856610.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  balance_weight = balance_weight.fillna(0)
C:\Users\misss\AppData\Local\Temp\ipykernel_40080\1845856610.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.00694' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  balance_weight.loc[tics, date] = w
C:\Users\misss\AppData\Local\Temp\ipykernel_40080\1845856610.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0069' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  

In [26]:
#calculate equally weighted allocation portolio
(balance_share_equal,
 balance_cost_equal,
 balance_cash_equal,
 portfolio_equal,
 portfolio_cost_equal,
 portfolio_return_equal,
 portfolio_cumsum_equal)=cal_portfolio(stocks_name,tradedate_full, weight_equal, capital, transaction_percent)

C:\Users\misss\AppData\Local\Temp\ipykernel_40080\1845856610.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  balance_weight = balance_weight.fillna(0)
C:\Users\misss\AppData\Local\Temp\ipykernel_40080\1845856610.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.006944444444444444' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  balance_weight.loc[tics, date] = w
C:\Users\misss\AppData\Local\Temp\ipykernel_40080\1845856610.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.006896551724137931' has dtype incompatible with int64, please explicitly cast to a

In [27]:
sum(portfolio_return_equal)

np.float64(173.68927657704597)

# 3. Calculate SPX Portfolio Value, Return and Cumulative Return

In [28]:
import yfinance as yf
import pandas as pd

spx = yf.download("^GSPC", start="1999-09-01", end="2025-12-31", interval="1d")
spx.reset_index(inplace=True)
spx.columns = spx.columns.droplevel(1)
spx = spx[['Date','Close']]
spx.rename(columns={'Date':'date','Close':'close'}, inplace=True)
spx.to_csv("SPX.csv")
print("S&P 500 data saved successfully.")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

S&P 500 data saved successfully.


In [29]:
import datetime

dates = pd.date_range('1999-09-01', '2025-12-31')


In [30]:
spx = pd.read_csv("SPX.csv")


In [31]:
spx.date = pd.to_datetime(spx.date)

In [32]:
spx = spx.set_index("date").reindex(dates, method='ffill')

In [33]:
tradedate = tradedate[:-1]

In [34]:

spx_daily = list(set(spx.index))
spx_daily.sort()
spx = spx['close']
spx.index = spx.index.astype(str)
spx = spx[tradedate]
spx.index = range(len(spx))
spx_return = (spx - spx.shift(1))/spx
spx_return[0] = 0

spx_return.index = tradedate
spx_return.index = range(len(spx))
spx_cumsum=spx_return.copy()

spx_value = spx_return.copy()
initial = 0
for i in range(len(spx_cumsum)):
    spx_cumsum[i] = initial + spx_cumsum[i]
    initial = spx_cumsum[i]

initial = 1000000
for i in range(len(spx_value)):
    spx_value[i] = initial * (1 + spx_value[i])
    initial = spx_value[i]

spx_cumsum.index = tradedate
spx_value.index = tradedate

In [35]:
sum(spx_return)

1.2693761168484916

# 4. Plot the Portfolio Performance

## 4.1 Plot Portfolio Value with Transaction Cost

In [36]:
time_ind = tradedate_full[:-1]

In [37]:
trace0 = go.Scatter(x = time_ind, y = portfolio_cost_meanv, mode = 'lines', name = 'Mean-Variance')
trace1 = go.Scatter(x = time_ind, y = portfolio_cost_equal, mode = 'lines', name = 'Equally Weighted')
trace2 = go.Scatter(x = time_ind, y = portfolio_cost_minv, mode = 'lines', name = 'Min-Variance')
trace3 = go.Scatter(x = time_ind, y = spx_value, mode = 'lines', name = 'SPX')

data = [trace0, trace1, trace2, trace3]
layout = dict(title = "Portfolio Value Performance Quarterly with Transaction cost", 
              xaxis = dict(title = 'Quarterly Trade Date'), 
              yaxis = dict(title = 'Portfolio Value'))
fig = dict(data = data, layout = layout)

plotly.offline.iplot(fig)

#portfolio_tmp = portfolio_cost_value

## 4.2 Plot Portfolio Return

In [38]:
trace_return1 = go.Scatter(x = time_ind, y = portfolio_return_meanv, mode = 'lines', name = 'Mean-Variance')
trace_return2 = go.Scatter(x = time_ind, y = portfolio_return_minv, mode = 'lines', name = 'Minimum-Variance')
trace_return3 = go.Scatter(x = time_ind, y = portfolio_return_equal, mode = 'lines', name = 'Equally Weighted')
trace_return4 = go.Scatter(x = time_ind, y = spx_return, mode = 'lines', name = 'SPX')

data = [trace_return1,trace_return2,trace_return3, trace_return4]
layout = dict(title = "Portfolio Return Quarterly", 
              xaxis = dict(title = 'Quarterly Trade Date'), 
              yaxis = dict(title = 'Portfolio Value'))
fig = dict(data = data, layout = layout)
plotly.offline.iplot(fig)

## 4.3 Plot Portfolio Cumulative Return

In [39]:
trace_return1 = go.Scatter(x = time_ind, y = portfolio_cumsum_meanv, mode = 'lines', name = 'Mean-Variance')
trace_return2 = go.Scatter(x = time_ind, y = portfolio_cumsum_minv, mode = 'lines', name = 'Minimum-Variance')
trace_return3 = go.Scatter(x = time_ind, y = portfolio_cumsum_equal, mode = 'lines', name = 'Equally Weighted')
trace_return4 = go.Scatter(x = time_ind, y = spx_cumsum, mode = 'lines', name = 'SPX')

data = [trace_return1,trace_return2,trace_return3, trace_return4]
layout = dict(title = "Portfolio Cumulative Return Quarterly", 
              xaxis = dict(title = 'Quarterly Trade Date'), 
              yaxis = dict(title = 'Portfolio Value'))
fig = dict(data = data, layout = layout)
plotly.offline.iplot(fig)

# 5. Get all Quarterly Return and Calcualte Annulized return, standard deviation and Sharpe ratio

In [40]:
quarter_return = pd.DataFrame(index = range(len(portfolio_return_equal)), columns = ['Mean-Var', 'Equally', 'Min-Var'])
quarter_return['SPX'] = spx_return
quarter_return['Mean-Var'] = portfolio_return_meanv
quarter_return['Equally'] = portfolio_return_equal
quarter_return['Min-Var'] = portfolio_return_minv

#writer_return = pd.ExcelWriter('quarter_return_user8.xlsx')
quarter_return.index=tradedate_full
#quarter_return.to_excel(writer_return,'Sheet1')
#writer_return.save()

In [41]:
quarter_return.head()

,Mean-Var,Equally,Min-Var,SPX
2004-12-01,0,0,0,0.000000
2005-03-01,0.045243,0.044764,0.045243,-0.031011
2005-06-01,0.313088,0.313189,0.313865,-0.013826
2005-09-01,0.07979,0.080351,0.079975,0.071733
2005-12-01,0.083836,0.064506,0.064406,0.015730


In [42]:
annulizedR_spx=quarter_return['SPX'].mean()*4
annulizedR_meanv=quarter_return['Mean-Var'].mean()*4
annulizedR_equal=quarter_return['Equally'].mean()*4
annulizedR_minv=quarter_return['Min-Var'].mean()*4
annulizedS_spx=quarter_return['SPX'].std()*2
annulizedS_meanv=quarter_return['Mean-Var'].std()*2
annulizedS_equal=quarter_return['Equally'].std()*2
annulizedS_minv=quarter_return['Min-Var'].std()*2

In [43]:
print("SPX",annulizedR_spx,annulizedS_spx)
print("Mean-Var",annulizedR_meanv,annulizedS_meanv)
print("Equally",annulizedR_equal,annulizedS_equal)
print("Min-Var",annulizedR_minv,annulizedS_minv)


SPX 0.059124621378103294 0.17844413131058864
Mean-Var 8.548934727985962 30.900103883854406
Equally 8.577248226026962 30.920526532922292
Min-Var 8.575092965431466 30.9105713412925


In [44]:
risk_free=0.015

In [45]:
sharpe_spx=(annulizedR_spx-risk_free)/annulizedS_spx
sharpe_meanv=(annulizedR_meanv-risk_free)/annulizedS_meanv
sharpe_equal=(annulizedR_equal-risk_free)/annulizedS_equal
sharpe_minv=(annulizedR_minv-risk_free)/annulizedS_minv

In [46]:
print("SPX Sharpe",sharpe_spx)
print("Mean-Var Sharpe",sharpe_meanv)
print("Equally Sharpe",sharpe_equal)
print("Min-Var Sharpe",sharpe_minv)


SPX Sharpe 0.24727415272235964
Mean-Var Sharpe 0.27617818891686713
Equally Sharpe 0.2769114625816737
Min-Var Sharpe 0.27693092019933957


In [47]:
insample_date=tradedate

In [48]:
insample_date

['2004-03-01',
 '2004-06-01',
 '2004-09-01',
 '2004-12-01',
 '2005-03-01',
 '2005-06-01',
 '2005-09-01',
 '2005-12-01',
 '2006-03-01',
 '2006-06-01',
 '2006-09-01',
 '2006-12-01',
 '2007-03-01',
 '2007-06-01',
 '2007-09-04',
 '2007-12-03',
 '2008-03-03',
 '2008-06-02',
 '2008-09-02',
 '2008-12-01',
 '2009-03-02',
 '2009-06-01',
 '2009-09-01',
 '2009-12-01',
 '2010-03-01',
 '2010-06-01',
 '2010-09-01',
 '2010-12-01',
 '2011-03-01',
 '2011-06-01',
 '2011-09-01',
 '2011-12-01',
 '2012-03-01',
 '2012-06-01',
 '2012-09-04',
 '2012-12-03',
 '2013-03-01',
 '2013-06-03',
 '2013-09-03',
 '2013-12-02',
 '2014-03-03',
 '2014-06-02',
 '2014-09-02',
 '2014-12-01',
 '2015-03-02',
 '2015-06-01',
 '2015-09-01',
 '2015-12-01',
 '2016-03-01',
 '2016-06-01',
 '2016-09-01',
 '2016-12-01',
 '2017-03-01',
 '2017-06-01',
 '2017-09-01',
 '2017-12-01',
 '2018-03-01',
 '2018-06-01',
 '2018-09-04',
 '2018-12-03',
 '2019-03-01',
 '2019-06-03',
 '2019-09-03',
 '2019-12-02',
 '2020-03-02',
 '2020-06-01',
 '2020-09-

In [49]:
quarter_return['SPX'][insample_date]


KeyError: "['2004-03-01', '2004-06-01', '2004-09-01'] not in index"

In [ ]:
annulizedR_spx_insample=quarter_return['SPX'][insample_date].mean()*4
annulizedR_meanv_insample=quarter_return['Mean-Var'][insample_date].mean()*4
annulizedR_equal_insample=quarter_return['Equally'][insample_date].mean()*4
annulizedR_minv_insample=quarter_return['Min-Var'][insample_date].mean()*4
annulizedS_spx_insample=quarter_return['SPX'][insample_date].std()*2
annulizedS_meanv_insample=quarter_return['Mean-Var'][insample_date].std()*2
annulizedS_equal_insample=quarter_return['Equally'][insample_date].std()*2
annulizedS_minv_insample=quarter_return['Min-Var'][insample_date].std()*2
print("SPX",annulizedR_spx_insample,annulizedS_spx_insample)
print("Mean-Var",annulizedR_meanv_insample,annulizedS_meanv_insample)
print("Equally",annulizedR_equal_insample,annulizedS_equal_insample)
print("Min-Var",annulizedR_minv_insample,annulizedS_minv_insample)
sharpe_spx_insample=(annulizedR_spx_insample-risk_free)/annulizedS_spx_insample
sharpe_meanv_insample=(annulizedR_meanv_insample-risk_free)/annulizedS_meanv_insample
sharpe_equal_insample=(annulizedR_equal_insample-risk_free)/annulizedS_equal_insample
sharpe_minv_insample=(annulizedR_minv_insample-risk_free)/annulizedS_minv_insample
print("SPX Sharpe",sharpe_spx_insample)
print("Mean-Var Sharpe",sharpe_meanv_insample)
print("Equally Sharpe",sharpe_equal_insample)
print("Min-Var Sharpe",sharpe_minv_insample)

SPX 0.06773879078983287 0.2750324039850313
Mean-Var 0.2769817423850113 0.30413300949795075
Equally 0.2815628824134959 0.31130453935360886
Min-Var 0.2815590917443039 0.31123331296013934
SPX Sharpe 0.19175482607025168
Mean-Var Sharpe 0.8614051556504145
Equally Sharpe 0.8562768887565395
Min-Var Sharpe 0.8564606700004602
